<a href="https://colab.research.google.com/github/Chia-Yin-Lee/NTHU/blob/main/TextFeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title show your CPU or GPU details
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8158043093438203145
 xla_global_id: -1]

In [ ]:
#@title connect google drive folder

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/thesis

Mounted at /content/drive
/content/drive/MyDrive/thesis


In [ ]:
#@title Useful Packages

import gensim
import gensim.downloader as api
from gensim.utils import simple_preprocess

import collections
import pickle
#import zipfile

import random
import numpy as np
import math
import os
from os import listdir
from os.path import isfile, join

from nltk import RegexpTokenizer
from nltk.corpus import stopwords

import pandas as pd
import smart_open
from nltk import bigrams
from queue import PriorityQueue
import operator
import re, collections
from collections import defaultdict, Counter

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)

# below is for visualization
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.decomposition import PCA

In [ ]:
word_vectors = api.load("word2vec-google-news-300")  # load pre-trained word-vectors from gensim-data

In [ ]:
#@title Prepare the Training and Test Data
# Set file names for train and test data
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_cp = list(read_corpus(lee_train_file))
test_cp = list(read_corpus(lee_test_file, tokens_only=True))

In [ ]:
#@title Prepare for training!

# init model
doc_m = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
# build vocab
doc_m.build_vocab(train_cp)
# Training
doc_m.train(train_cp, total_examples=doc_m.corpus_count, epochs=doc_m.epochs)

In [ ]:
#@title Inferring

sent1 = 'thousands of customers reporting issues with DBS’ services in the afternoon'
sent2 = 'DBS customers could not use their online and mobile services on Saturday afternoon'
sent1 = sent1.split()
sent2 = sent2.split()
sims = doc_m.similarity_unseen_docs(sent1, sent2)
print(sims)

0.6535897


In [ ]:
df = pd.read_csv("TheBusinessTimes_reindex.csv", index_col=False)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#@title Utility functions for reading and preprocessing data

MAX_LEN = 15

def read_corpus(doc):
  line = doc.strip()
  items = line.split('\t')
  en_sent = preprocess(items[0])
  return en_sent

def preprocess(sent):
    tokens = []
    tokens = gensim.utils.simple_preprocess(sent, min_len=1)  # This lowercases, tokenizes, de-accents (optional)
    return tokens

read_corpus(sentence)

In [ ]:
df['Headline_token'] = None
df['Contents_token'] = None

for i in range(len(df)):

  df['Headline_token'][i] = list(simple_preprocess(df['Headline'][i]))
  df['Contents_token'][i] = list(simple_preprocess(df['Contents'][i]))

df.head()

In [ ]:
#@title Inferring doc to vec
df['Headline_vec'] = None
df['Contents_vec'] = None

for i in range(len(df)):
  df['Headline_vec'][i] = doc_m.infer_vector(df['Headline_token'][i])
  df['Contents_vec'][i] = doc_m.infer_vector(df['Contents_token'][i])

df.head()

In [ ]:
df.to_csv("TheBusinessTimes_Doc2Vec.csv", index = False)

## Doc2Vec w/o pre-trained model

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv("TheBusinessTimes_Doc2Vec.csv", index_col=False)

In [ ]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
from nltk.tokenize import word_tokenize

In [ ]:
df_headline = df['Headline'].tolist()
df_contents = df['Contents'].tolist()

In [ ]:
#@title Take the average & median length of a list of sentences
def avg_median_len(sentences):

  total_length = 0
  num_sentences = len(sentences)

  for sentence in sentences:
      words = sentence.split()
      total_length += len(words)

  average_length = total_length / num_sentences

  print(f"Total length of all sentences: {total_length} words")
  print(f"Number of sentences: {num_sentences}")
  print(f"Average sentence length: {average_length:.2f} words")

  # Calculate the sentence lengths (number of words) and sort them
  sentence_lengths = [len(sentence.split()) for sentence in sentences]
  sentence_lengths.sort()

  if num_sentences % 2 == 0:
      # If there's an even number of sentences, the median is the average of the two middle values
      median_length = (sentence_lengths[num_sentences // 2 - 1] + sentence_lengths[num_sentences // 2]) / 2
  else:
      # If there's an odd number of sentences, the median is the middle value
      median_length = sentence_lengths[num_sentences // 2]

  print(f"Median sentence length: {median_length} words")
  print('---')

avg_median_len(df_headline)
avg_median_len(df_contents)

Total length of all sentences: 7407 words
Number of sentences: 508
Average sentence length: 14.58 words
Median sentence length: 12.0 words
---
Total length of all sentences: 365362 words
Number of sentences: 508
Average sentence length: 719.22 words
Median sentence length: 595.5 words
---


According to the research of Kim et al. (2019), the text vector dimension of Doc2Vec usually chooses a value slightly lower than the average length of the document, so we chose 10 and 300 for 'Headline' and 'Contents', respectively.

Too large or too small sliding windows can affect the performance of the model, so we chose 2 and 10 as the candidate values of the parameter w (Kim et al., 2017).

DM has no sliding window parameters, so three hyper-parameters are combined to produce six different scenarios for Doc2Vec parameter optimization.

In [ ]:
doc_headline = [TaggedDocument(words=word_tokenize(text.lower()), tags=[str(i)]) for i, text in enumerate(df_headline)]
doc_contents = [TaggedDocument(words=word_tokenize(text.lower()), tags=[str(i)]) for i, text in enumerate(df_contents)]

# Create and train a Doc2Vec model for Headline
model_headline = Doc2Vec(vector_size=10, window=2, min_count=1, dm=1, epochs=100)
model_headline.build_vocab(doc_headline)
model_headline.train(doc_headline, total_examples=model_headline.corpus_count, epochs=model_headline.epochs)

vectorized_headline = [model_headline.infer_vector(word_tokenize(text.lower())) for text in df_headline]

# Create and train a Doc2Vec model for Contents
model_contents = Doc2Vec(vector_size=300, window=10, min_count=1, dm=1, epochs=100)
model_contents.build_vocab(doc_contents)
model_contents.train(doc_contents, total_examples=model_contents.corpus_count, epochs=model_contents.epochs)

vectorized_contents = [model_contents.infer_vector(word_tokenize(text.lower())) for text in df_contents]


1. Preprocessed the text data using NLTK's word_tokenize function and convert it to lowercase
2. Created TaggedDocument objects to prepare the data for training the Doc2Vec model
3. Created and train a Doc2Vec model with specific parameters such as vector size, window size, and others
4. Used the model it infer vectors for the input text data

In [ ]:
df['Headline_vec1'] = vectorized_headline
df['Contents_vec1'] = vectorized_contents
df.to_csv("TheBusinessTimes_Doc2Vec_1.csv", index = False)

## Dimensionality Reduction

After obtaining text feature vectors using Doc2Vec, we can further reduce their dimensionality using different methods.

In [ ]:
df = pd.read_csv("TheBusinessTimes_Doc2Vec_1.csv", index_col=False)
df_sae = df[['Date', 'Headline_vec', 'Contents_vec','Headline_vec1', 'Contents_vec1']]
df_sae.head()

,Date,Headline_vec,Contents_vec,Headline_vec1,Contents_vec1
0,2022-04-11,[-0.00467339 -0.04872593 0.06713308 -0.036835...,[ 1.37918100e-01 9.95347381e-01 7.51885593e-...,[-0.5500672 1.7635964 -0.41078466 0.039500...,[-3.1550055e+00 -5.0798702e-01 -1.7155163e+00 ...
1,2022-04-12,[ 2.67287605e-02 -4.76982296e-02 8.35237950e-...,[ 0.0114288 1.0734031 0.7698896 0.054947...,[-0.54261017 1.8881685 -0.3878786 0.022236...,[-3.1653442 -0.48148593 -1.6370666 1.417602...
2,2022-04-13,[ 0.03323345 -0.06435372 0.0567309 -0.008780...,[ 3.34855579e-02 1.01118553e+00 7.15755105e-...,[-0.51115805 1.7620329 -0.3719802 0.088263...,[-3.1744192e+00 -6.9273657e-01 -1.7152363e+00 ...
3,2022-04-14,[ 1.18304482e-02 -2.10055280e-02 8.28568172e-...,[ 4.82800007e-01 4.34585541e-01 6.22129083e-...,[ 0.9779878 0.47930837 0.96984106 -0.801769...,[-2.4194320e-01 -1.7393562e+00 2.9316941e-01 ...
4,2022-04-15,[ 1.16834119e-02 -3.47062536e-02 2.15828419e-...,[ 0.40390685 0.43239427 0.69738793 -0.224561...,[ 0.8195557 0.36460224 0.7548852 -0.922970...,[-8.9547247e-02 -1.3415108e+00 1.8680838e-01 ...


In [ ]:
#@title strlist_to_list function

def strlist_to_list(strlist):
  a = strlist.split(' ')
  a = [item.strip("[]\n") for item in a]
  a = [item for item in a if item != ""]
  a = [float(item) for item in a]
  return a


for col in df_sae.columns[1:]:
    for index, value in df_sae[col].iteritems():
      df_sae.at[index, col] = strlist_to_list(value)

type(df_sae['Headline_vec'].iloc[2])

<ipython-input-6-c7d102fab4a1>:12: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, value in df_sae[col].iteritems():


list

In [ ]:
#@title Method 1: PCA

from sklearn.decomposition import PCA

def PCA_1col(dataframe, col_name, n):

  df = dataframe[col_name]
  X = np.array(df.tolist())

  # Initialize and fit the PCA model to reduce dimension to 5
  pca = PCA(n_components=n)
  X_reduced = pca.fit_transform(X)

  # Create a new DataFrame with the reduced dimension features
  df_reduced = pd.DataFrame(X_reduced, columns=[f'PCA_{col_name}_{i+1}' for i in range(n)])

  return df_reduced

df_PCA_0 = pd.concat([PCA_1col(df_sae, 'Headline_vec', 5), PCA_1col(df_sae, 'Contents_vec', 20)], axis=1)
df_PCA_1 = pd.concat([PCA_1col(df_sae, 'Headline_vec1', 5), PCA_1col(df_sae, 'Contents_vec1', 20)], axis=1)

In [ ]:
#@title Method 2: SVD

from sklearn.decomposition import TruncatedSVD

def SVD_1col(dataframe, col_name, n):

  df = dataframe[col_name]
  X = np.array(df.tolist())

  # Initialize and fit the PCA model to reduce dimension to 5
  svd = TruncatedSVD(n_components=n)
  X_reduced = svd.fit_transform(X)

  # Create a new DataFrame with the reduced dimension features
  df_reduced = pd.DataFrame(X_reduced, columns=[f'SVD_{col_name}_{i+1}' for i in range(n)])

  return df_reduced

df_SVD_0 = pd.concat([SVD_1col(df_sae, 'Headline_vec', 5), SVD_1col(df_sae, 'Contents_vec', 20)], axis=1)
df_SVD_1 = pd.concat([SVD_1col(df_sae, 'Headline_vec1', 5), SVD_1col(df_sae, 'Contents_vec1', 20)], axis=1)

In [ ]:
#@title Method 3: SAE
import tensorflow as tf
from tensorflow.keras import layers, models

def SAE_1col(dataframe, col_name, n, m1, m2):
  df = dataframe[col_name]
  input_data = np.array(df.tolist()) # list of arrays

  # Define the autoencoder model
  input_dim = input_data.shape[1]  # Input dimension
  output_dim = n  # Output dimension

  if m2 > 0:
    # Encoder
    encoder = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(m1, activation='relu'),
        layers.Dense(m2, activation='relu'),
        layers.Dense(output_dim, activation='sigmoid')
    ])

    # Decoder
    decoder = models.Sequential([
        layers.Input(shape=(output_dim,)),
        layers.Dense(m2, activation='relu'),
        layers.Dense(m1, activation='relu'),
        layers.Dense(input_dim, activation='sigmoid')
    ])
  else:
    # Encoder
    encoder = models.Sequential([
        layers.Input(shape=(input_dim,)),
        layers.Dense(m1, activation='relu'),
        layers.Dense(output_dim, activation='sigmoid')
    ])

    # Decoder
    decoder = models.Sequential([
        layers.Input(shape=(output_dim,)),
        layers.Dense(m1, activation='relu'),
        layers.Dense(input_dim, activation='sigmoid')
    ])

  # Combine encoder and decoder
  autoencoder = models.Sequential([encoder, decoder])

  # Compile the model
  autoencoder.compile(optimizer='adam', loss='mse')  # loss function

  # Train the autoencoder
  autoencoder.fit(input_data, input_data, epochs=100, batch_size=32)  # Adjust the number of epochs and batch size

  # Transform the data using the encoder
  X_reduced = encoder.predict(input_data)

  # Create a new DataFrame with the reduced dimension features
  df_reduced = pd.DataFrame(X_reduced, columns=[f'SAE_{col_name}_{i+1}' for i in range(n)])

  return df_reduced


df_SAE_0 = pd.concat([SAE_1col(df_sae, 'Headline_vec', 5, 100, 25), SAE_1col(df_sae, 'Contents_vec', 20, 100, 50)], axis=1)
df_SAE_1 = pd.concat([SAE_1col(df_sae, 'Headline_vec1', 5, 8, 0), SAE_1col(df_sae, 'Contents_vec1', 20, 100, 50)], axis=1)

Epoch 1/100
16/16 [==============================] - 1s 4ms/step - loss: 0.2349
Epoch 2/100
16/16 [==============================] - 0s 3ms/step - loss: 0.1479
Epoch 3/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0451
Epoch 4/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0131
Epoch 5/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0088
Epoch 6/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0078
Epoch 7/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0074
Epoch 8/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0071
Epoch 9/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0069
Epoch 10/100
16/16 [==============================] - 0s 4ms/step - loss: 0.0067
Epoch 11/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0066
Epoch 12/100
16/16 [==============================] - 0s 3ms/step - loss: 0.0066
Epoch 13/100
16/16 [=================

In [ ]:
df_SAE_0.head()

,SAE_Headline_vec_1,SAE_Headline_vec_2,SAE_Headline_vec_3,SAE_Headline_vec_4,SAE_Headline_vec_5,SAE_Contents_vec_1,SAE_Contents_vec_2,SAE_Contents_vec_3,SAE_Contents_vec_4,SAE_Contents_vec_5,...,SAE_Contents_vec_11,SAE_Contents_vec_12,SAE_Contents_vec_13,SAE_Contents_vec_14,SAE_Contents_vec_15,SAE_Contents_vec_16,SAE_Contents_vec_17,SAE_Contents_vec_18,SAE_Contents_vec_19,SAE_Contents_vec_20
0,0.000068,0.999991,0.999997,0.999991,0.999998,0.993805,0.999982,0.000441,0.999952,0.982941,...,0.999441,0.024678,0.999970,0.999335,0.634667,0.999644,0.999996,0.999754,0.999998,0.896413
1,0.000011,0.999999,1.000000,0.999999,1.000000,0.994223,0.999978,0.000421,0.999944,0.978622,...,0.999448,0.020851,0.999963,0.999425,0.612444,0.999552,0.999995,0.999689,0.999997,0.918437
2,0.000013,0.999999,1.000000,0.999999,1.000000,0.993927,0.999978,0.000460,0.999946,0.980721,...,0.999396,0.022860,0.999966,0.999323,0.625085,0.999565,0.999995,0.999704,0.999997,0.901601
3,0.000146,0.999983,0.999994,0.999982,0.999995,0.924747,0.999036,0.010962,0.997907,0.981480,...,0.993783,0.357244,0.998697,0.976758,0.258638,0.993813,0.999771,0.998373,0.999846,0.571318
4,0.000109,0.999987,0.999996,0.999987,0.999997,0.947366,0.999233,0.009652,0.997904,0.980305,...,0.994063,0.315910,0.998884,0.981579,0.320820,0.994369,0.999800,0.998363,0.999879,0.656073


In [ ]:
#@title Save the new dataframes

df_PCA_0.to_csv("TheBusinessTimes_PCA_0.csv", index = False)
df_PCA_1.to_csv("TheBusinessTimes_PCA_1.csv", index = False)
df_SVD_0.to_csv("TheBusinessTimes_SVD_0.csv", index = False)
df_SVD_1.to_csv("TheBusinessTimes_SVD_1.csv", index = False)
df_SAE_0.to_csv("TheBusinessTimes_SAE_0.csv", index = False)
df_SAE_1.to_csv("TheBusinessTimes_SAE_1.csv", index = False)

## Class PreProcessor

In [ ]:
# Combine the lists of words into a single list with repeated words merged
#merged_words = list(set().union(*df['Headline_token'], *df['Contents_token']))
merged_words = pd.concat([df['Headline_token'], df['Contents_token']]).tolist()

print(merged_words)
print(len(merged_words))

In [ ]:
merged_words = [x for i, x in enumerate(merged_words) if x not in merged_words[:i]]

print(merged_words)

[['data', 'centre', 'reits', 'committed', 'to', 'sustainable', 'growths', 'reits', 'punching', 'above', 'their', 'weight', 'in', 'the', 'global', 'arena'], ['not', 'the', 'kingmaker', 'says', 'sabana', 'reit', 'director', 'whom', 'quarz', 'wants', 'voted', 'outproperty', 'listcos', 'have', 'more', 'women', 'chair', 'than', 'those', 'in', 'other', 'sectors'], ['time', 'to', 'double', 'down', 'on', 'reit', 'investments', 'amid', 'mas', 'double', 'barrel', 'policy', 'tightening'], ['border', 'reopening', 'is', 'godsend', 'to', 'singapore', 'retail', 'space', 'landlordsgreen', 'commercial', 'logistics', 'properties', 'valuations', 'more', 'resilient', 'ireus'], ['sustainability', 'linked', 'loans', 'trump', 'other', 'green', 'finance', 'for', 'singapore', 'property', 'firmsfct', 'raises', 'dpu', 'by', 'on', 'improved', 'operating', 'financial', 'performance'], ['cuscaden', 'offers', 'per', 'sph', 'reit', 'unit', 'below', 'last', 'traded', 'priceelite', 'commercial', 'reit', 'dpu', 'up', 'o

In [ ]:
merged_words_list = []
for wl in merged_words:
  for w in wl:
    merged_words_list.append(w)
print(merged_words_list)
print(len(merged_words_list))

['data', 'centre', 'reits', 'committed', 'to', 'sustainable', 'growths', 'reits', 'punching', 'above', 'their', 'weight', 'in', 'the', 'global', 'arena', 'not', 'the', 'kingmaker', 'says', 'sabana', 'reit', 'director', 'whom', 'quarz', 'wants', 'voted', 'outproperty', 'listcos', 'have', 'more', 'women', 'chair', 'than', 'those', 'in', 'other', 'sectors', 'time', 'to', 'double', 'down', 'on', 'reit', 'investments', 'amid', 'mas', 'double', 'barrel', 'policy', 'tightening', 'border', 'reopening', 'is', 'godsend', 'to', 'singapore', 'retail', 'space', 'landlordsgreen', 'commercial', 'logistics', 'properties', 'valuations', 'more', 'resilient', 'ireus', 'sustainability', 'linked', 'loans', 'trump', 'other', 'green', 'finance', 'for', 'singapore', 'property', 'firmsfct', 'raises', 'dpu', 'by', 'on', 'improved', 'operating', 'financial', 'performance', 'cuscaden', 'offers', 'per', 'sph', 'reit', 'unit', 'below', 'last', 'traded', 'priceelite', 'commercial', 'reit', 'dpu', 'up', 'on', 'maiden

### Class PreProcessor

It takes the tokenized corpus as inputs, and:

1. count each word frequency in the corpus
2. build vocabulary *w2id_vocab*, *id2w_vocab*, and *count_vocab*, note that 1) a special token 'UNK' should be added as 0-index for out-of-vocabulary word, 2) the vocabulary size should be smaller than the pre-defined *max_vocabulary_size*, and 3) the minimum count of words in your vocabulary should be larger than the pre-defined *min_occurrence*. Example:


```
w2id_vocab = {'UNK':0, 'a':1, ...}  key denotes the word, value denotes the index in your vocabulary, a.k.a. id.

id2w_vocab = {0:'UNK', 1:'a', ...} key denotes the id, value denotes the word.

count_vocab - {0:10000, 1:2000, ...} key denotes the id, value denotes the counts.
```

3. replace each word in inputs with its corresponding id. For word not included in your vocabulary, use id 0 (i.e., 'UNK') instead.
4. return the list of word ids as *data*.

In [ ]:
#@title class for preparing data for neural network
class PreProcessor():

    def __init__(self):

        self.model_args = {
            'max_vocabulary_size': 50000,
            'min_occurrence': 5,  # Remove all words that does not appears at least n times.
            'num_negatives': 25,
            'window_size': 3,
            'sample_t': 0.00001,
            'neg_alpha': 0.75,
            # Training Parameters.
            'emb_size': 50,
            'learning_rate': 0.1,
            'batch_size': 128,
            'num_steps': 50
        }

        self.w2id_vocab = {}
        self.id2w_vocab = {}
        self.count_vocab = {}
        self.neg_probs = {}
        self.sub_probs = {}

        self.pretrained_emb = None
        self.vocab_size = len(self.w2id_vocab)
        self.data_index = 0

    def save(self, path):
        f = open(path, 'wb')  # binary format for writing
        pickle.dump(self, f)
        f.close()

    def load(self, path):
        f = open(path, 'rb')  # binary format for reading
        proc = pickle.load(f)
        f.close()
        return proc

    def set_model_arg(self, key, value):
        self.model_args[key] = value

    def get_model_arg(self, key):
        return self.model_args.get(key, None)

    def get_vocab_size(self):
        self.vocab_size = len(self.w2id_vocab)
        return self.vocab_size

    def words2ids(self, words):
        word_ids = list()
        for w in words:
            wid = self.w2id_vocab.get(w, 0)
            word_ids.append(wid)
        return word_ids

    def id2words(self, word_ids):
        words = list()
        for wid in word_ids:
            w = self.id2w_vocab.get(wid, self.id2w_vocab[0])
            words.append(w)
        return words

    def convert_texts_to_features(self, words):

        draftcount = {}
        for word in words:
            if word not in draftcount.keys():
                draftcount[word] = 1
            else:
                draftcount[word] += 1
        print(draftcount)
        self.w2id_vocab['UNK'] = 0
        self.id2w_vocab[0] = 'UNK'
        self.count_vocab[0] = 0

        id = 1
        for key in draftcount.keys():
            if len(self.w2id_vocab) <= self.model_args['max_vocabulary_size']:

                if draftcount[key] >= self.model_args['min_occurrence']:
                    self.w2id_vocab[key] = id
                    self.id2w_vocab[id] = key
                    self.count_vocab[id] = draftcount[key]
                    id += 1
                else:
                    self.count_vocab[0] += draftcount[key]
            else:
                break
        data = []
        for word in words:
            if word in self.w2id_vocab.keys():
                data.append(self.w2id_vocab[word])
            else:
                data.append(0)

        print("Words count: {}".format(len(words)))
        print("Unique words: {}".format(len(draftcount)))
        print("Vocabulary size: {}".format(len(self.count_vocab)))

        return data  # return the list of word ids as data

    def count_sampling_probs(self):
        # build negtive sampling and sub sampling probs
        total_num = sum([c for c in self.count_vocab.values()])
        total_weight_num = sum([pow(c, self.get_model_arg('neg_alpha')) for c in self.count_vocab.values()])
        for w, c in self.count_vocab.items():
            self.neg_probs[w] = self.get_neg_prob(c, total_weight_num)
            self.sub_probs[w] = self.get_sub_prob(c, total_num)

    def load_pretrained_model(self, model):
        self.pretrained_emb = []
        vec_size = len(word_vectors.__getitem__('hello'))
        for id, w in self.id2w_vocab.items():
            vec = model.__getitem__(w) if model.__contains__(w) else np.zeros(vec_size)
            self.pretrained_emb.append(vec)
        self.pretrained_emb = torch.FloatTensor( np.array(self.pretrained_emb) )

    def get_neg_prob(self, count, total):
        return pow(count, self.get_model_arg('neg_alpha'))/total

    def get_sub_prob(self, count, total):
        inverse_prob = self.get_model_arg('sample_t')*total/count
        return (math.sqrt(1/inverse_prob) + 1) * inverse_prob

    def get_batch(self, data):

        batch_size = self.get_model_arg('batch_size')
        window_size = self.get_model_arg('window_size')

        batch_count = 0
        batch = []
        pos_label = []
        neg_label = []

        while True:
            tmp_window = random.randint(1, window_size)
            # get window size (words left and right + current one).
            span = 2 * tmp_window + 1
            if self.data_index + span >= len(data):
                self.data_index = 0
            text_span = data[self.data_index:self.data_index + span]
            self.data_index += 1

            if batch_count == 0 :
                batch = []
                pos_label = []
                neg_label = []
                batch_count = 0

            current_word = text_span[tmp_window]
            for i, w in enumerate(text_span):
              if i == tmp_window: continue
              # skip frequent words by subsampling
              sub_prob = self.sub_probs[w]
              if random.random() > sub_prob : continue
              batch.append(current_word)
              pos_label.append(w)
              # negative sampling
              neg_label.append( np.random.choice(list(self.neg_probs.keys()), size=self.get_model_arg('num_negatives'), p=list(self.neg_probs.values()), replace=True) )
              batch_count += 1
              if batch_count == self.get_model_arg('batch_size'):
                  res_batch = batch
                  res_pos = pos_label
                  res_neg = neg_label

                  batch_count = 0
                  batch = []
                  pos_label = []
                  neg_label = []
                  yield torch.LongTensor( np.array(res_batch) ), torch.LongTensor(np.array(res_pos)), torch.LongTensor(np.array(res_neg))

In [ ]:
# preprare dataset and hyper-parameters for training
proc = PreProcessor()

data = proc.convert_texts_to_features(merged_words_list)
proc.count_sampling_probs()
proc.load_pretrained_model(word_vectors)

# arguments for reproduction
proc.set_model_arg('init_seed', 42)
proc.set_model_arg('log_step', 10)
proc.set_model_arg('checkpoint_path', "./word2vec.bin")

# save proc
arg_path = "./proc.dat"
proc.save(arg_path)

{'data': 145, 'centre': 88, 'reits': 871, 'committed': 35, 'to': 3380, 'sustainable': 17, 'growths': 1, 'punching': 1, 'above': 53, 'their': 282, 'weight': 4, 'in': 3122, 'the': 6791, 'global': 165, 'arena': 1, 'not': 191, 'kingmaker': 1, 'says': 4, 'sabana': 189, 'reit': 1818, 'director': 18, 'whom': 2, 'quarz': 90, 'wants': 5, 'voted': 19, 'outproperty': 1, 'listcos': 2, 'have': 450, 'more': 271, 'women': 12, 'chair': 3, 'than': 210, 'those': 52, 'other': 131, 'sectors': 57, 'time': 82, 'double': 24, 'down': 91, 'on': 1340, 'investments': 36, 'amid': 85, 'mas': 14, 'barrel': 3, 'policy': 24, 'tightening': 14, 'border': 9, 'reopening': 67, 'is': 916, 'godsend': 1, 'singapore': 748, 'retail': 278, 'space': 79, 'landlordsgreen': 1, 'commercial': 239, 'logistics': 216, 'properties': 226, 'valuations': 61, 'resilient': 50, 'ireus': 12, 'sustainability': 30, 'linked': 21, 'loans': 49, 'trump': 1, 'green': 56, 'finance': 15, 'for': 1528, 'property': 385, 'firmsfct': 1, 'raises': 7, 'dpu': 3

In [ ]:
print(proc.id2words(data[:20]))

['data', 'centre', 'reits', 'committed', 'to', 'sustainable', 'UNK', 'reits', 'UNK', 'above', 'their', 'UNK', 'in', 'the', 'global', 'UNK', 'not', 'the', 'UNK', 'UNK']


In [ ]:
data_iter = proc.get_batch(data)
center_words, pos_words, neg_words = next(data_iter)
print(proc.id2words(center_words.numpy()[:5]))
print(proc.id2words(pos_words.numpy()[:5]))
print(proc.id2words(neg_words.numpy()[0][:5]))

['reits', 'the', 'quarz', 'UNK', 'UNK']
['sustainable', 'not', 'wants', 'wants', 'wants']
['units', 'somerset', 'falling', 'keppel', 'market']


In [ ]:
proc.pretrained_emb

tensor([[-0.0947, -0.5117,  0.0581,  ...,  0.3906, -0.0034,  0.2812],
        [-0.1729, -0.1426,  0.0437,  ..., -0.1348,  0.1001, -0.1650],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [-0.2471,  0.1982, -0.0056,  ..., -0.1235,  0.1338, -0.2090],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1914,  0.1504, -0.2256,  ..., -0.0879, -0.2305, -0.0669]])

In [ ]:
print(len(proc.pretrained_emb), len(proc.pretrained_emb[0]))

2254 300


##Skip-gram with negative sampling by portch

In [ ]:
#@title vanilla skipgram
class SkipgramNegSampling(nn.Module):

    def __init__(self, vocab_size, embed_dim, pretrained_emb=None):
        super(SkipgramNegSampling, self).__init__()

        self.embedding_i = nn.Embedding(vocab_size, embed_dim)
        self.embedding_o = nn.Embedding(vocab_size, embed_dim)

        nn.init.xavier_normal_(self.embedding_i.weight)
        nn.init.xavier_normal_(self.embedding_o.weight)

        self.logsigmoid = nn.LogSigmoid()
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    def forward(self, center_words, pos_words, neg_words):

        center_vectors = self.embedding_i(center_words)
        pos_vectors = self.embedding_o(pos_words)
        n_samples = len(neg_words[0])

        batch_size, embed_size = center_vectors.shape
        center_vectors = center_vectors.view(batch_size, embed_size, 1)   # batch of column vectors
        pos_vectors = pos_vectors.view(batch_size, 1, embed_size) # batch of row vectors
        neg_vectors = self.embedding_o(neg_words).view(batch_size, n_samples, embed_size)

        # log-sigmoid loss for correct pairs
        out_loss = self.logsigmoid(torch.bmm(pos_vectors, center_vectors)).squeeze()

        # log-sigmoid loss for incorrect pairs
        neg_loss = self.logsigmoid(torch.bmm(neg_vectors.neg(), center_vectors))
        neg_loss = neg_loss.squeeze().sum(1)  # sum the losses over the sample of noise vectors
        loss = -(out_loss + neg_loss).mean()  # average batch loss

        return loss
    def similarity(self, inputs, targets):
        input_embeds = self.embedding_i(inputs) # B x D
        target_embeds = self.embedding_i(targets) # B x D
        sim_score = self.cos(input_embeds, target_embeds) # Bx1
        return sim_score

In [ ]:
#@title utility functions
# some of the codes are revised from https://github.com/312shan/Pytorch-seq2seq-Beam-Search/blob/master/model.py

def check_gpu():
    # torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
    is_cuda = torch.cuda.is_available()

    # If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
    if is_cuda:
        device = torch.device("cuda")
        print("GPU is available")
    else:
        device = torch.device("cpu")
        print("GPU not available, CPU used")
    return device
def set_seed(seed=42):
    '''good for reproduction'''
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path):
    model.load_state_dict(torch.load(path))
    return model

In [ ]:
#@title training function

def train(model, proc, data, device):
    # fetch hyper-parameters
    batch_size = proc.get_model_arg("batch_size")
    num_steps = proc.get_model_arg('num_steps')
    learning_rate = proc.get_model_arg("learning_rate")
    checkpoint_path = proc.get_model_arg("checkpoint_path")
    log_step = proc.get_model_arg("log_step")

    data_iter = proc.get_batch(data)
    # Define Loss, Optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # train!
    for step in range(num_steps):
      model.train()
      optimizer.zero_grad() # Clears existing gradients from previous epoch
      # prepare inputs
      word, pos, neg = next(data_iter)

      loss = model(word.to(device), pos.to(device), neg.to(device))

      loss.backward() # Does backpropagation and calculates gradients
      # clip gradients to prevent exploding gradients
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1)

      optimizer.step() # Updates the weights accordingly
      if step % 10000 == 0:
          scheduler.step()

      if step % log_step == 0:
          print("\rstep: {}/{}, Loss: {:.4f}, saving model to {}".format(step, num_steps, loss.item(), proc.get_model_arg('checkpoint_path')), end=' ')


In [ ]:
# get parameters from preprocessor
init_seed = proc.get_model_arg('init_seed')

device = check_gpu()
set_seed(init_seed)

# Instantiate the model with hyperparameters
model = SkipgramNegSampling(vocab_size=proc.get_vocab_size(), embed_dim=proc.get_model_arg('emb_size'), pretrained_emb=proc.pretrained_emb)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

train(model, proc, data, device)

GPU not available, CPU used
step: 40/50, Loss: 8.1588, saving model to ./word2vec.bin 

In [ ]:
model.eval()

inputs = proc.words2ids(['good', 'growth'])
targets = proc.words2ids(['great', 'falling'])

sim_scores = model.similarity(torch.LongTensor(inputs).to(device), torch.LongTensor(targets).to(device))
print(sim_scores.detach().cpu().numpy())

[ 0.3418235  -0.21814153]


# SKIP THE LAST

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into a training set and a testing set
train_data, test_data = train_test_split(df['Contents'], test_size=0.2, random_state=42)

# Now you have 'train_data' and 'test_data' for unsupervised learning or text analysis tasks

In [ ]:
def read_corpus(df, tokens_only=False):
      for i, line in enumerate(df):
          tokens = gensim.utils.simple_preprocess(line)
          if tokens_only:
              yield tokens
          else:
              # For training data, add tags
              yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(train_data))
test_corpus = list(read_corpus(test_data, tokens_only=True))

In [ ]:
print(train_corpus[:2])

[TaggedDocument(words=['analysts', 'remain', 'positive', 'on', 'commercial', 'real', 'estate', 'investment', 'trusts', 'reits', 'in', 'singapore', 'and', 'malaysia', 'this', 'is', 'amid', 'expectations', 'that', 'interest', 'rate', 'volatility', 'will', 'persist', 'in', 'the', 'near', 'term', 'even', 'as', 'the', 'federal', 'reserve', 'slows', 'its', 'rate', 'hikes', 'rhb', 'on', 'friday', 'downgraded', 'singapore', 'listed', 'reits', 'reits', 'to', 'neutral', 'from', 'overweight', 'as', 'it', 'expects', 'rising', 'interest', 'rate', 'volatility', 'to', 'continue', 'in', 'the', 'near', 'term', 'from', 'an', 'operational', 'standpoint', 'reits', 'will', 'remain', 'resilient', 'rhb', 'said', 'it', 'expects', 'modest', 'cap', 'rate', 'expansion', 'of', 'between', 'five', 'and', 'basis', 'points', 'and', 'does', 'not', 'foresee', 'any', 'large', 'drops', 'in', 'asset', 'value', 'that', 'being', 'said', 'the', 'research', 'team', 'projects', 'distribution', 'per', 'unit', 'growth', 'to', 'p

In [ ]:
print(test_corpus[:2])

[['in', 'to', 'date', 'the', 'straits', 'times', 'index', 'sti', 'has', 'generated', 'an', 'per', 'cent', 'decline', 'in', 'total', 'returns', 'while', 'the', 'ftse', 'developed', 'index', 'declined', 'per', 'cent', 'the', 'iedge', 'reit', 'index', 'maintained', 'resilience', 'with', 'per', 'cent', 'decline', 'in', 'total', 'returns', 'while', 'the', 'ftse', 'epra', 'nareit', 'developed', 'index', 'benchmark', 'for', 'global', 'reits', 'declined', 'per', 'cent', 'for', 'the', 'quarter', 'to', 'date', 'singapore', 'stocks', 'have', 'seen', 'billion', 'of', 'net', 'institutional', 'outflows', 'reducing', 'the', 'year', 'to', 'date', 'ytd', 'net', 'institutional', 'inflow', 'to', 'million', 'the', 'technology', 'sector', 'has', 'continued', 'to', 'weigh', 'on', 'global', 'stocks', 'with', 'hospitality', 'and', 'real', 'estate', 'also', 'lagging', 'banks', 'technology', 'and', 'reits', 'have', 'now', 'booked', 'the', 'most', 'net', 'institutional', 'outflows', 'across', 'the', 'sectors', '

In [ ]:
#@title Prepare for training!

# init model
doc_model = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=40)
# build vocab
doc_model.build_vocab(train_corpus)
print(f"Word 'market' appeared {doc_model.wv.get_vecattr('market', 'count')} times in the training corpus.")

Word 'market' appeared 707 times in the training corpus.


In [ ]:
#@title training!
doc_model.train(train_corpus, total_examples=doc_model.corpus_count, epochs=doc_model.epochs)

In [ ]:
#@title Inferring
sent1 = ['a', 'book', 'is', 'a', 'ladder', 'of', 'human', 'progress']
vec1 = doc_model.infer_vector(sent1)
print(vec1)

In [ ]:
sent2 = ['books', 'are', 'the', 'means', 'to', 'our', 'growth']
sims = doc_model.similarity_unseen_docs(sent1, sent2)
print(sims)

0.109275356


In [ ]:
from nltk.tokenize import word_tokenize

# Tokenize and preprocess the text data
df['Tokens'] = df['Contents'].apply(lambda x: word_tokenize(x.lower()))  # Lowercase for consistency

In [ ]:
from gensim.models.doc2vec import TaggedDocument

tagged_data = [TaggedDocument(words=doc, tags=[str(i)]) for i, doc in enumerate(df['Tokens'])]


In [ ]:
#@title Initialize and train the Doc2Vec model

from gensim.models import Doc2Vec

doc2vec_model = Doc2Vec(vector_size=300,  # Should match the dimension of your pre-trained word embeddings
                       window=5,
                       min_count=1,
                       workers=4,
                       epochs=20)

doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [ ]:
df.head()

,Date,Headline,Contents,AW9U,C2PU,HMN,J85,Q5T,ACV,BMGU,...,A17U,DHLU,BWCU,J91U,BUOU,AJBU,ME8U,M44U,M1GU,Tokens
0,2022-04-11,Data centre S-Reits committed to sustainable g...,THE data centre industry has experienced treme...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[the, data, centre, industry, has, experienced..."
1,2022-04-12,Data centre S-Reits committed to sustainable g...,THE data centre industry has experienced treme...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[the, data, centre, industry, has, experienced..."
2,2022-04-13,Data centre S-Reits committed to sustainable g...,THE data centre industry has experienced treme...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[the, data, centre, industry, has, experienced..."
3,2022-04-14,"I'm not the kingmaker,' says Sabana Reit direc...",SABANA Industrial Real Estate Investment Trust...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[sabana, industrial, real, estate, investment,..."
4,2022-04-15,"I'm not the kingmaker,' says Sabana Reit direc...",SABANA Industrial Real Estate Investment Trust...,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[sabana, industrial, real, estate, investment,..."


# Web Scraping: The Business Times

## Steps:

1. Extracting the links, title, news type, premium, and published date for each news
2. Browse each link extracted and extract the contents
3. Loop to next page
4. Back to Step 1
5. When reached the last page, break the loop

In [ ]:
url = "https://www.businesstimes.com.sg/keywords/s-reits?selector=.js-view-dom-id-c11bb3b103123a39ee279e7b7a37e3f502b60ac227b706e83146ad5e7d2dbc6c&page=0"

r = rq.get(url)
print(r)  # 200: success
soup = BeautifulSoup(r.text,"lxml")
print(soup)

<Response [200]>
<!DOCTYPE html>
<html>
<head>
<title></title>
<style>
    html, body {
      margin: 0;
      padding: 0;
      background-color: white;
    }
  </style>
</head>
<body>
<script>(function j(F){var Ip={},IH={};var Ih=ReferenceError,IC=TypeError,ID=Object,IK=RegExp,Iz=Number,IT=String,IV=Array,Ie=ID.bind,q=ID.call,Ik=q.bind(Ie,q),b=ID.apply,IR=Ik(b),Q=[].push,A=[].pop,M=[].slice,g=[].splice,L=[].join,N=[].map,a=Ik(Q),R=Ik(M),J=Ik(L),t=Ik(N),H={}.hasOwnProperty,Y=Ik(H),G=JSON.stringify,U=ID.getOwnPropertyDescriptor,In=ID.defineProperty,r=IT.fromCharCode,S=Math.min,II=Math.floor,Iv=ID.create,E="".indexOf,f="".charAt,X=Ik(E),Ii=Ik(f),Iw=typeof Uint8Array==="function"?Uint8Array:IV;var w=["IVWLfZoB8klu4Tf0","set","RS_COJV9","value","x4Fm51HNPqDSF6R-","b-I6my-uBLbSWdFPl-HVwow","k02rS64_-UZTwQyi","uuoixi-bRMvbHJpG0uW6x8aileY3pu-bxg","nzjqf-dr1Xo","unescape","EIFGhQ3qQr3vRuk","a02_FbY4lCk","capture","FyfTAYxw","lzPAb99n9Tkw","ChvWCdls_g418zvM","5pRR-UfdWbu1QJFmubCEjbA","eT_7cPV1

In [ ]:
from lxml import html

page = rq.get(url)
content = html.fromstring(page.content)
endpoints = content.xpath('//html/body/table[4]/tbody/tr/td/p[4]/text()')
'''
for endpoint in endpoints:
    r = rq.get(endpoint)
    data = r.json()[0]
    for article in data['chunks']:
       print (url, "\t", article['url'], "\t", article['title'])  '''

In [ ]:
#@title get_details & item_search functions

def get_details(url):
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'html.parser')

    content = soup.find('div', {'id':'story-body'})
    if content:
        content.get_text(strip=True)
        return content
    else:
        return ""


def item_search(item, limit, page):
    stocknews = f"https://www.thestar.com.my/search/?q={item}&qsort=oldest&qrec={limit}&qstockcode=&pgno={page}"

    html = requests.get(stocknews).text

    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
#@title star_new_crawler function and initialization
def star_new_crawler(page, search_query, limit):

    title = []
    links = []
    premium = []
    new_type = []
    contents = []
    publishedDate = []
    while True:
        print(page)
        try:
            result = item_search(search_query, limit, page)
            title += [x.get_text(strip=True) for x in result.find_all("h2", {"class": "f18"})]
            links += [x.find('a', {"data-content-type": "Article"})['href'] for x in
                     result.find_all("h2", {"class": "f18"})]
            premium += [x.get_text(strip=True) for x in result.find_all("span", {"class": "biz-icon"})]
            new_type += [x.get_text(strip=True) for x in result.find_all("a", {"class": "kicker"})]
            contents += [get_details(x) for x in links]
            publishedDate += [x.get_text(strip=True) for x in result.find_all("span", {"class": "timestamp"})]

            if len(title) == 0:
                break
        except Exception as e:
            print(e)
            traceback.print_exc()

        page += 1

    pd.DataFrame({'new_type': new_type, 'title': title, 'premium': premium, 'links': links, 'published_data': publishedDate,
                 'contents': contents}).to_excel(f'{search_query}.xlsx', index=False)

if __name__ == '__main__':
    page = 1
    search_query = 'AAPL'
    limit = 30
    star_new_crawler(page, search_query, limit)

Obtain list of news from the coverpage

In [ ]:
#@title url

url = "https://www.businesstimes.com.sg/keywords/s-reits?selector=.js-view-dom-id-c11bb3b103123a39ee279e7b7a37e3f502b60ac227b706e83146ad5e7d2dbc6c&page=0"

# Request
r1 = rq.get(url)
r1.status_code

# We'll save in coverpage the cover page content
coverpage = r1.content

# Soup creation
soup1 = BeautifulSoup(coverpage, 'html.parser')

headlines = []
headline_elements = soup.find_all('h2', class_='card-title')

for element in headline_elements:
    headline = element.text.strip()
    headlines.append(headline)

for headline in headlines:
    print(headline)
# News identification
#coverpage_news = soup1.find_all('h3', class_='card-title')
#len(coverpage_news)

[]
